In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy.stats import zscore
from sklearn.model_selection import train_test_split

In [53]:
try:
        data = pd.read_csv("train.csv")
        print(data.describe(),"\n")
except Exception as e:
        print("Error! data not  found.")

            pixel_0      pixel_1       pixel_2       pixel_3       pixel_4  \
count  50000.000000  50000.00000  50000.000000  50000.000000  50000.000000   
mean     130.710740    130.14036    131.050440    131.568860    132.184700   
std       73.412873     72.44259     72.240546     72.016555     71.714551   
min        0.000000      0.00000      0.000000      0.000000      0.000000   
25%       71.000000     71.00000     73.000000     73.000000     75.000000   
50%      128.000000    127.00000    129.000000    130.000000    130.000000   
75%      189.000000    188.00000    188.000000    188.000000    189.000000   
max      255.000000    255.00000    255.000000    255.000000    255.000000   

            pixel_5       pixel_6       pixel_7       pixel_8      pixel_9  \
count  50000.000000  50000.000000  50000.000000  50000.000000  50000.00000   
mean     132.851840    133.371540    133.890920    134.485040    134.93260   
std       71.537505     71.353558     71.281237     71.071698  

In [ ]:
try:
        test = pd.read_csv("test.csv")
        print(test.describe(),"\n")
except Exception as e:
        print("Error! data not  found.")

In [ ]:
X = data.drop(columns=['label'])
y = data['label']

In [ ]:
def check_missing_values():
    missing = X.isnull().sum()
    missing = missing[missing > 0]
    if not missing.empty:
        print(missing)
    else:
        print("\n No Missing Data \n")

def detect_outliers(z_thresh=3.29):
    for col in X.columns:
        if pd.api.types.is_numeric_dtype(X[col]):
            z_scores = zscore(X[col])
            outliers = (abs(z_scores) > z_thresh)
            num_outliers = outliers.sum()
            print(f"col '{col}': {num_outliers} outlier found.")
            #filtered_data = data[np.abs(z_scores) <= z_thresh]
def normal(X):
    return X.reshape(X.shape[0], -1).astype(np.float32) / 255.0

def plot_distributions():
    for col in X_train.columns:
        if pd.api.types.is_numeric_dtype(X_train[col]):
            plt.figure(figsize=(6,4))
            sns.histplot(X_train[col], kde=True)
            plt.title(f"Distribution of {col}")
            plt.show()
        else:
          print("\n",col, "is not numeric")

In [ ]:
check_missing_values()
detect_outliers()

In [ ]:
y= np.where(data['label'] == '0', '1', '0')
print(y)

In [ ]:
X = X.to_numpy().reshape(X.shape[0], -1)
X = normal(X)
X

In [ ]:
X_train, X_test,y_train,y_test = train_test_split(
    X,y,
    test_size=0.2,
    random_state=42,
)
print(X_train)

In [ ]:
y = y.to_numpy()  


#X = X.reshape(-1, 3, 32, 32).transpose(0, 2, 3, 1)
# plt.imshow(X[0])
# plt.title(f'Label: {y[0]}')
# plt.show()

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
y = np.array(y).astype(np.float32)

In [ ]:
def sigmoid_activation_function(z):
        return sigmoid(z) * (1 - sigmoid(z))

In [ ]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [ ]:
def binary_cross_entropy(y_true, y_pred):
    epsilon = 1e-8 
    bce = -np.mean(y_true * np.log(y_pred + epsilon) + (1 - y_true) * np.log(1 - y_pred + epsilon))
    return bce

# bce_loss = binary_cross_entropy(y_true, y_pred)
# print(f"Binary Cross-Entropy Loss (manual calculation): {bce_loss}")

# # Compute Binary Cross-Entropy using Keras
# bce_loss_keras = binary_crossentropy(y_true, y_pred).numpy()
# print(f"Binary Cross-Entropy Loss (Keras): {bce_loss_keras}")

In [ ]:
# Creating the Feed forward neural network
def f_forward(x, w1, w2):
	# hidden
	z1 = x.dot(w1)    # input from layer 1 
	a1 = sigmoid(z1)  # out put of layer 2 
	z2 = a1.dot(w2)   # input of out layer
	a2 = sigmoid(z2)  # output of out layer
	return(a2)

# initializing the weights randomly
def generate_wt(x, y):
	li =[]
	for i in range(x * y):
		li.append(np.random.randn())
	return(np.array(li).reshape(x, y))
	
# Back propagation of error 
def back_prop(x, y, w1, w2, alpha):
	
	# hidden layer
	z1 = x.dot(w1)
	a1 = sigmoid(z1) 
	z2 = a1.dot(w2)
	a2 = sigmoid(z2)
	
	# error in output layer
	d2 =(a2-y)
	d1 = np.multiply((w2.dot((d2.transpose()))).transpose(), 
								(np.multiply(a1, 1-a1)))
	# Gradient for w1 and w2
	w1_adj = x.transpose().dot(d1)
	w2_adj = a1.transpose().dot(d2)
	
	# Updating parameters
	w1 = w1-(alpha*(w1_adj))
	w2 = w2-(alpha*(w2_adj))
	
	return(w1, w2)

In [ ]:
def forward(X, y, W, b):
    z = np.dot(X,W)+ b
    y_pred = sigmoid(z)
    loss = binary_cross_entropy(y, y_pred)
    return z, loss, y_pred
    
def backward(X, y, W, b, lr, loss, y_pred):
    dz = y_pred - y
    dw = np.dot(X.T,dz)/X.shape[0]
    db= np.mean(dz)

    W  -= lr * dw
    b -= lr * db

    return W, b
def params(X):
    W = np.random.normal(0, 1/np.sqrt(X.shape[1]), size=X.shape[1])
    b = 0
    return W,b

def train(X, y, epochs=100, lr=0.1):
    W , b = params(X)
    last_loss=0
    loss=10
    for epoch in range(epochs):
        last_loss = loss
        z, loss, ypred =forward(X, y, W, b)
        W , b = backward(X, y, W, b, lr, loss, ypred)
        if epoch % 10 == 0:
            print(f"Epoch {epoch} - Loss: {loss:.4f}")
        # if (last_loss-loss)<0.0000001:
        #     break
    return W , b
        
         

In [ ]:
def predict(X, W, b):
    z = np.dot(X, W) + b
    y_pred = sigmoid(z)
    return (y_pred >= 0.5).astype(int)


In [ ]:
def evaluate(y_true, y_pred):
    TP = np.sum((y_true == 1) & (y_pred == 1))
    TN = np.sum((y_true == 0) & (y_pred == 0))
    FP = np.sum((y_true == 0) & (y_pred == 1))
    FN = np.sum((y_true == 1) & (y_pred == 0))

    precision = TP / (TP + FP + 1e-8)
    recall = TP / (TP + FN + 1e-8)
    f1_score = 2 * precision * recall / (precision + recall + 1e-8)

    print("Confusion Matrix:")
    print(f"TP: {TP}, FP: {FP}")
    print(f"FN: {FN}, TN: {TN}")
    print(f"F1-score: {f1_score}")


In [ ]:
W , b= train(X_train, y_train, 50, 0.01)
y_pred = predict(X_test, W, b)
evaluate(y_test, y_pred)

In [ ]:
X_test

In [127]:
y.shape

(50000,)

In [128]:
y = np.array(y).astype(np.float32)

In [129]:
def sigmoid_activation_function(z):
        return sigmoid(z) * (1 - sigmoid(z))

In [130]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [131]:
def binary_cross_entropy(y_true, y_pred):
    epsilon = 1e-8 
    bce = -np.mean(y_true * np.log(y_pred + epsilon) + (1 - y_true) * np.log(1 - y_pred + epsilon))
    return bce

# bce_loss = binary_cross_entropy(y_true, y_pred)
# print(f"Binary Cross-Entropy Loss (manual calculation): {bce_loss}")

# # Compute Binary Cross-Entropy using Keras
# bce_loss_keras = binary_crossentropy(y_true, y_pred).numpy()
# print(f"Binary Cross-Entropy Loss (Keras): {bce_loss_keras}")

In [132]:
# Creating the Feed forward neural network
def f_forward(x, w1, w2):
	# hidden
	z1 = x.dot(w1)    # input from layer 1 
	a1 = sigmoid(z1)  # out put of layer 2 
	z2 = a1.dot(w2)   # input of out layer
	a2 = sigmoid(z2)  # output of out layer
	return(a2)

# initializing the weights randomly
def generate_wt(x, y):
	li =[]
	for i in range(x * y):
		li.append(np.random.randn())
	return(np.array(li).reshape(x, y))
	
# Back propagation of error 
def back_prop(x, y, w1, w2, alpha):
	
	# hidden layer
	z1 = x.dot(w1)
	a1 = sigmoid(z1) 
	z2 = a1.dot(w2)
	a2 = sigmoid(z2)
	
	# error in output layer
	d2 =(a2-y)
	d1 = np.multiply((w2.dot((d2.transpose()))).transpose(), 
								(np.multiply(a1, 1-a1)))
	# Gradient for w1 and w2
	w1_adj = x.transpose().dot(d1)
	w2_adj = a1.transpose().dot(d2)
	
	# Updating parameters
	w1 = w1-(alpha*(w1_adj))
	w2 = w2-(alpha*(w2_adj))
	
	return(w1, w2)

In [138]:
def forward(X, y, W, b):
    z = np.dot(X,W)+ b
    y_pred = sigmoid(z)
    loss = binary_cross_entropy(y, y_pred)
    return z, loss, y_pred
    
def backward(X, y, W, b, lr, loss, y_pred):
    dz = y_pred - y
    dw = np.dot(X.T,dz)/X.shape[0]
    db= np.mean(dz)

    W  -= lr * dw
    b -= lr * db

    return W, b
def params(X):
    W = np.random.normal(0, 1/np.sqrt(X.shape[1]), size=X.shape[1])
    b = 0
    return W,b

def train(X, y, epochs=100, lr=0.1):
    W , b = params(X)
    last_loss=0
    loss=10
    for epoch in range(epochs):
        last_loss = loss
        z, loss, ypred =forward(X, y, W, b)
        W , b = backward(X, y, W, b, lr, loss, ypred)
        if epoch % 10 == 0:
            print(f"Epoch {epoch} - Loss: {loss:.4f}")
        # if (last_loss-loss)<0.0000001:
        #     break
    return W , b
        
         

In [139]:
def predict(X, W, b):
    z = np.dot(X, W) + b
    y_pred = sigmoid(z)
    return (y_pred >= 0.5).astype(int)


In [142]:
def evaluate(y_true, y_pred):
    TP = np.sum((y_true == 1) & (y_pred == 1))
    TN = np.sum((y_true == 0) & (y_pred == 0))
    FP = np.sum((y_true == 0) & (y_pred == 1))
    FN = np.sum((y_true == 1) & (y_pred == 0))

    precision = TP / (TP + FP + 1e-8)
    recall = TP / (TP + FN + 1e-8)
    f1_score = 2 * precision * recall / (precision + recall + 1e-8)

    print("Confusion Matrix:")
    print(f"TP: {TP}, FP: {FP}")
    print(f"FN: {FN}, TN: {TN}")
    print(f"F1-score: {f1_score}")


In [141]:
W , b= train(X_train, y_train, 50, 0.01)
y_pred = predict(X_test, W, b)
evaluate(y_test, y_pred)

Epoch 0 - Loss: 1.9123
Epoch 10 - Loss: -64.3131
Epoch 20 - Loss: -64.3135
Epoch 30 - Loss: -64.3135
Epoch 40 - Loss: -64.3135
Confusion Matrix:
TP: 979, FP: 973
FN: 0, TN: 0
F1-score: 0.6680


In [137]:
X_test

array([[0.7254902 , 0.7019608 , 0.7372549 , ..., 0.4392157 , 0.4392157 ,
        0.44313726],
       [0.84313726, 0.827451  , 0.80784315, ..., 0.42352942, 0.42352942,
        0.4627451 ],
       [0.654902  , 0.6392157 , 0.6392157 , ..., 0.48235294, 0.43529412,
        0.43529412],
       ...,
       [0.5647059 , 0.57254905, 0.5803922 , ..., 0.6117647 , 0.6039216 ,
        0.5921569 ],
       [0.1764706 , 0.14117648, 0.13725491, ..., 0.16078432, 0.16862746,
        0.13725491],
       [0.3254902 , 0.3764706 , 0.38039216, ..., 0.34117648, 0.3647059 ,
        0.38431373]], dtype=float32)